# Part 1

### Table showing reads and index relative to file name

| read1 | read2 | index1 | index2|
|-------|-------|--------|-------|
|1294_S1_L008_R1_001.fastq.gz| 1294_S1_L008_R4_001.fastq.gz|1294_S1_L008_R2_001.fastq.gz|1294_S1_L008_R3_001.fastq.gz|

##### Script shown below to create tables that were then plotted in R to create distributions

In [ ]:
#!/usr/bin/env python3
#make sure to change add import gzip, file, change gzip, change output file name

import numpy as np
import gzip

###Set up function to convert scores to phred scores###
def convert_phred(letter):
    """Converts a single character into a phred score based on Illumina 1.8+Phred+33"""
    ASCII = ord(letter) -33
    return(ASCII)
    
###Create a numpy array to contain all scores for each pos###
all_qscores_array = np.zeros((101),dtype=float)             
#now populate array for first position
#print(all_qscores_array)


#file = "/projects/bgmp/shared/2017_sequencing/1294_S1_L008_R1_001.fastq.gz"
#file = "/projects/bgmp/shared/2017_sequencing/1294_S1_L008_R2_001.fastq.gz"
#file = "/projects/bgmp/shared/2017_sequencing/1294_S1_L008_R3_001.fastq.gz"
file = "/projects/bgmp/shared/2017_sequencing/1294_S1_L008_R4_001.fastq.gz"

#file2 = "/projects/bgmp/jlee33/demulti_dir/table_R1"
#file2 = "/projects/bgmp/jlee33/demulti_dir/table_R2"
#file2 = "/projects/bgmp/jlee33/demulti_dir/table_R3"
file2 = "/projects/bgmp/jlee33/demulti_dir/table_R4"

LN = 0
#with open(file, "r") as fh, open(file2, "w") as out_fh:
with gzip.open(file, "rt") as fh, open(file2, "w") as out_fh:
    i = 0
    for line in fh:
        i+=1
        line  = line.strip("\n")
        if i%4==0:
            #print(line)
            #start a counter for how many sequences have processed(aka no. reads)
            pos = 0    #letter position on that given line
            #continue for the length of the line aka all things on lines
            for pos in range(len(line)):
                scores = convert_phred(line[pos]) #converts to character to phred score
                #print(pos, scores)
                #ADD THINGS TO ARRAY
                all_qscores_array[pos]+=scores
                
    #create and fill mean_scores
    mean_scores = np.zeros((101),dtype=float)
    
    #numpy.array/___ means divide each index by ___
    mean_scores = all_qscores_array/(i/4)
    
   # print(mean_scores)
    out_fh.write("# Base Pair\tMean Quality Score\n")
    pos = 0
    for pos in range(len(mean_scores)):
        #output of .write can only print strings so convert int varaibles
        out_fh.write(str(pos) + "\t" + str(mean_scores[pos]) + "\n")

### See pdf in repo 
for per base call distribution of quality scores of read 1, read 2, index 1, index2


### What is a good quality score cut off for index reads and pairs to utilize for sample identification and downstream analysis, respectively?

This is a subjective cutoff. The standard quality cutoff is 30 because this contains 99.99% of data. Furthermore as a general rule of thumb, it is best to visualize the data first to see general trends. From this, pick a threshold value that occurs when there is a substantial dip in average quality score. For instance, the average quality score gets worse as the read progresses, so pick a quality score value that is reached at a steep decline on the curve.

Based on the distributions of my two reads (R1 and R4), the quality score cutoff will be:
    
    qual sc cutoff = 30 (scores 30 and higher will be kept)

### How many indexes have Undetermined (N) base calls? (Utilize your command line
tool knowledge. Submit the command you used. CHALLENGE: use a one-line
command)

##### Script shown below

In [ ]:
#this code produces two files that contain only the lines that contain N in seq
####Index 1####
zcat /projects/bgmp/shared/2017_sequencing/1294_S1_L008_R2_001.fastq.gz | grep -A1 "^@" | grep -v "^--" | grep -v "^@" | grep "N" | wc -l 

####Index 2#### 
zcat /projects/bgmp/shared/2017_sequencing/1294_S1_L008_R3_001.fastq.gz | grep -A1 "^@" | grep -v "^--" | grep -v "^@" | grep "N" | wc -l

then counted number of lines which is number of indexes with N

    wc -l /projects/bgmp/jlee33/demulti_dir/N_index1

    output = 3976613 N_index1

then counted number of lines which is number of indexes with N
wc -l /projects/bgmp/jlee33/demulti_dir/N_index2

    output = 3328051 N_index2
    
then add numbers together

    Total is: 7304664

# Part 2
Write a strategy for writing an algorithm for de-multiplexing files and reporting indexhopping. That is, given four files (2 with biological reads, 2 with index reads) and known indexes, sort reads by index, outputting one forward file and one reverse file per index, plus a pair of files for unknown indexes. Additionally, your algorithm should report the number of properly matched indexes (per index) and the level of index hopping observed. You should not write any code for this portion of the assignment.

##### Outline

Problem: Illumina output includes a mixture of several barcodes (24 different samples with 24 different barcodes) and need to separate barcodes out for downstream applications (i.e. analysis)

Goal: Demultiplex data so each sample (aka index) are moved into files separate from other indexes. 

Output for Goal 1: Sort further so that each index has fastq output files == 1) forward reads 2) reverse reads 3) low quality reads 4) reads with one or more unknown sequences

Goal 2: Analysis of level of index swapping and undertermined pairs before and after quality filetering of index reads

Output for Goal2: output number of properly matched indexes (per index) and the level of index hopping observed

In [ ]:
#create a file that contains barcodes (tab deliniated)
    #so column 1 has short and column two has barcode
        #B1    GTAGCGTA
        #A5    CGATCGAT
        #etc for all barcodes
#create dictionary that has all 24 expected barcoes
    #index_dict that has column1 as key and barcode as value of above file

#create comp_base_dict that contains A or T or G or C as key and the value is the complimentary base of key
comp_base_dict = {}

#create variables to refer to each file as F (forward read), I1 (index 1), I2 (index2), R (reverse read)
F = "test_1294_S1_L008_R1_001.fastq"
I1 = "test_1294_S1_L008_R2_001.fastq"
I2 = "test_1294_S1_L008_R3_001.fastq"
R = "test_1294_S1_L008_R4_001.fastq"


#create function to read the forward index to get the reverse complimentary barcode
def convert_to_reverse_index:
    '''takes forward index sequence and outputs the reverse complimentary sequence (aka other barcode)'''
    with open(I1, "r") as in_fh:
        #rename each lines as header, seq, plus, qual
        while True:
            header equals in_hf.readline().strip()
            if not header:
                break   #tells to end when reach last line in file
            seq equals in_fh.readline().strip()
            plus equals in_fh.readline().strip()
            qual equals in_fh.readline().strip()
            
            if seq in comp_base_dict #validates that barcode is one of the 24 we expected
                look up each nucleotide in I1 as a key in comp_base_dict and print value aka the complimantery base
                then reverse order of printed out seq (aka the reverse complimentary barcode)
                save this string as reverse_comp_index   
            else statement for if index doesn't match any of the 24 expected index
                print("index found that shouldn't be here")
    return reverse_comp_index
  
#Create convert_phred function
def convert_phred(letter):
    """Converts a single character into a phred score based on Illumina 1.8+Phred+33"""
    ASCII = ord(letter) -33
    return(ASCII)

In [ ]:
#create output forward and reverse for each read that is correct. 
#then two more (one for forward and one for reverse)
    #each of those have 1) index hop 2) low qual 3)N

def filter_reads ():
    '''filters out reads with unknown sequences (have Ns) and low quality reads (<37 phred score) and index hopping. Makes new file of reads with no index hopping. Returns total_correct_indexed, total_incorrect_indexed, total_low_quality, total_with_N'''
    with open the files F, I1, I2, and R as read
    with open to create new output files as write (unacceptable_forward_reads.fastq and unacceptable_reverse_reads.fastq) along with acceptable_forward_reads and acceptable_reverse_reads
    with open to create new output file as write (index_hop)
        while True:
            #tells program to read fastq files 4 lines at a time (header, seq, plus, qual) for ALL input files
            header = in_fh.readline().strip()
            if not header:
                break   #tells to end when reach last line in file
            seq = in_fh.readline().strip()
            plus = in_fh.readline().strip()
            qual = in_fh.readline().strip()

            #create counters of correct and incorrectly indexed samples
                #correct means index from forward/reverse are the same
            total_correct_indexed = 0
            total_index_hop = 0
            total_low_quality = 0
            total_with_N = 0
    
    ###Step 1: get rid of reads with low quality or unknown sequences###
            #convert ASCII to phred using convert_phred and variable qual
            #name this value as variable "score"
            score = convert_phred(letter)

            #if/else statement for looking at sequences with N
            #if N is present then add read info to unknown_reads file
            if "n" or "N" in qual of index1 fastq file or index2 fastq file:
                #perform the follwoing write commands to write out info contained in index1 and forward read (F)
                unacceptable_forward_reads.write(header) 
                unacceptable_forward_reads.write(seq)
                unacceptable_forward_reads.write(plus)
                unacceptable_forward_reads.write(qual)
                #perform the follwing write commands to write out info contained in index2 and reverse read (R)
                unacceptable_reverse_reads.write(header)
                unacceptable_reverse_reads.write(seq)
                unacceptable_reverse_reads.write(plus)
                unacceptable_reverse_reads.write(qual)
                #add to total N counter
                total_with_N+=1
            
            #if no N (all nucleotides are determined) then continue to 
            #step that considers if qual scores are above threshold
            else:
                qual_thres == 30 #create variable "qual_thres" that is a subjective number that
                #quality scores less than this will be not used
                
                #if statement: score < #qual_thres then write to file "unacceptable_forward_reads and unnacetalbe_reverse_reads"
                if score < qual_thres for seq of any of the input files (read1, index1, read2, index2):
                    #perform the follwing write commands to write out info contained in index1 and forward read (F)
                    unaccept_forward.write(header)
                    unaccept_forward.write(seq)
                    unaccept_forward.write(plus)
                    unaccept_forward.write(qual)
                    #perform the follwing write commands to write out info contained in index2 and reverse read (R) 
                    unaccept_reverse.write(header)
                    unaccept_reverse.write(seq)
                    unaccept_reverse.write(plus)
                    unaccept_reverse.write(qual)
                    #add to counter
                    total_low_quality+=1

                    #else: score is = or > "qual thres" then add to forward or reverse file
                    else:
                    ###Step 2: determine if index swapping occured###
                        #if not then means index from somewhere else appeared
                        now looking at index2 fastq(I2)
                        if seq not in index_dict:
                            print("Index not on list is found")
                            break
                        else seq in index_dict:
                            continue
                
                    #compare index on forward/reverse read to see if 
                    #they are reverse complimentary

                        created_index = #function

                        #compare created_index to index from index2 file 
                        if seq of index2 fastq file == convert_reverse_index(of index1 fastq file)
                            #perform the follwing write commands to write out info contained in index2 and forward read (F)
                            acceptable_forward_reads.write(header)
                            acceptable_forward_reads.write(seq)
                            acceptable_forward_reads.write(plus)
                            acceptable_forward_reads.write(qual)
                            #perform the follwing write commands to write out info contained in index2 and reverse read (R)
                            acceptable_reverse_reads.write(header)
                            acceptable_reverse_reads.write(seq)
                            acceptable_reverse_reads.write(plus)
                            acceptable_reverse_reads.write(qual)
                            #add to counter
                            total_correct_indexed+=1
                        else: #when indexes are not reverse complimentary (aka index hopping occured)
                            #perform the follwing write commands to write out info contained in index 1, index2, forward read (F) and reverse read (R)
                            index_hop.write(header)
                            index_hop.write(seq)
                            index_hop.write(plus)     
                            index_hop.write(qual)
                            total_index_hop+=1
    ###program then loops back and repeats for all lines in file
    return (total_correct_indexed, total_index_hop, total_low_quality, total_with_N)

#### Unit tests 
using test examples for individual functions

In [2]:
#convert_phred function
def convert_phred(letter):
    """Converts a single character into a phred score based on Illumina 1.8+Phred+33"""
    ASCII = ord(letter) -33
    return(ASCII)

assert convert_phred("A") == 32
assert convert_phred("B") == 33
assert convert_phred("T") == 51

In [3]:
##Create dictionary of known 24 indexed libraries
    #fill with the specific barcodes in that flow cell
index_dict = {"B1":"GTAGCGTA", "A5": "CGATCGAT", "C1": "GATCAAGG", "B9": "AACAGCGA", "C9": "TAGCCATG", "C3": "CGGTAATC", "B3": "CTCTGGAT", "C4": "TACCGGAT", "A11": "CTAGCTCA", "C7": "CACTTCAC", "B2": "GCTACTCT", "A1" : "ACGATCAG"}

assert index_dict["B1"] == "GTAGCGTA"
assert index_dict["A5"] == "CGATCGAT"
assert index_dict["C1"] == "GATCAAGG"